In [56]:
import os
import pandas as pd
from pandas import read_csv, read_excel, DataFrame
from sae import COTSCar
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum as qsum
import numpy as np
from typing import List

import sys
sys.path.append('../')
import saedfsc

Price of the car.

In [57]:
maxProductPrice = 400000

Information about customers

In [58]:
customers = saedfsc.customers['Name'].to_list()
cQty = dict(zip(saedfsc.customers['Name'], saedfsc.customers['Quantity'])) # customer quantities
cPriceFocus = dict(zip(saedfsc.customers['Name'], saedfsc.customers['PriceFocus']))
name_weights_dict = saedfsc.customers.set_index('Name')['PerformanceUtilityWeights'].to_dict()
cWts = {c : np.fromstring(name_weights_dict[c].strip('[]'), sep=',') for c in name_weights_dict}

Methods to help compute the market share for each customer type (based on logit model).

In [59]:
def getTotalUtilityForCustomer(car : COTSCar, c : str):
    total_utility = 0
    perf_utility = car.partworth_objectives(weights=cWts[c])[0]
    total_utility += (1-cPriceFocus[c])*perf_utility - cPriceFocus[c]*pricePerf
    return total_utility

def getMarketShare(car : COTSCar, c : str, competitors : List[COTSCar]): # based on logit model of demand
    carUtility = getTotalUtilityForCustomer(car, c)
    totalCompetitorUtility = sum([getTotalUtilityForCustomer(competitorCar, c) for competitorCar in competitors])
    return carUtility / (totalCompetitorUtility + carUtility)

Create a set of competitors.

In [60]:
profitMargin = 0.1
competitors = []
for i in range(3):
    competitorCar = COTSCar()
    competitorCar.price = (1+profitMargin)*competitorCar.cost()
    competitorCar.name = "Competitor Car " + str(i)
    competitors.append(competitorCar)

names = [car.name for car in competitors]
designs = [car.vector for car in competitors]
salesPrices = ['$' + format(car.price, ",.2f") for car in competitors]
costs = ['$' + format(car.cost(), ",.2f") for car in competitors]
competitorsDF = pd.DataFrame({
    'Name': names,
    'Design': designs,
    'SalesPrice': salesPrices,
    'ProductCost': costs
})
competitorsDF

,Name,Design,SalesPrice,ProductCost
0,Competitor Car 0,"[1, 0, 4, 7, 7, 0, 3, 5, 8, 3, 57, 106, 0, 2, ...","$5,055.17","$4,595.61"
1,Competitor Car 1,"[2, 6, 0, 9, 0, 0, 4, 15, 23, 1, 55, 98, 1, 2,...","$47,641.19","$43,310.17"
2,Competitor Car 2,"[5, 8, 6, 7, 4, 6, 1, 0, 13, 1, 28, 20, 1, 3, ...","$3,729.43","$3,390.39"


Create a random car and compute market share

In [63]:
carDesignVec = [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]
product = COTSCar()
product.vector = carDesignVec
product.price = (1+profitMargin)*product.cost()
print("OUR CAR:") 
print("Design: ", product.vector)
print("Sales price: $", format(product.price, ",.2f"))
print("Cost: $", format(product.cost(), ",.2f"))

utilities = [getTotalUtilityForCustomer(product, c) for c in customers]
market_shares = [getMarketShare(product, c, competitors) for c in customers]
expected_customer_demand = [getMarketShare(product, c, competitors)*cQty[c] for c in customers]

df = pd.DataFrame({
    'Name': customers,
    'Utility for our car': utilities,
    'Market share': market_shares,
    'Expected demand': expected_customer_demand
})

print("Market information for our car:")
df

OUR CAR:
Design:  [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]
Sales price: $ 579,003.25
Cost: $ 526,366.60
Market information for our car:


,Name,Utility for our car,Market share,Expected demand
0,CustomerType1,-0.312363,0.252507,52.269038
1,CustomerType2,-2.103368,0.248990,90.134529
2,CustomerType3,-1.697901,0.275016,15.400872
3,CustomerType4,-0.728397,0.249158,70.262419
4,CustomerType5,-1.391240,0.255106,158.675999
5,CustomerType6,-0.873645,0.249929,90.724361
6,CustomerType7,-2.817649,0.256616,252.766335
7,CustomerType8,-0.733528,0.246665,188.945216
8,CustomerType9,-1.795050,0.255831,29.676362
9,CustomerType10,-0.857968,0.307947,234.963833
